# Reading input data and building the SFS

Now that our models are set up, we are ready to read out input data and convert it into site frequency spectra.

In [1]:
from delimitpy import parse_input
from delimitpy import process_empirical
import numpy as np
import os
import pickle

## Parse configuration files and read in intermediate files from previous part of tutorial.

We need to read our config file into memory again, and we need to read the files that were created in the 'Building a Model Set' notebook.

The test data we are using was simulated under a model with three populations and no gene flow. 

In [2]:
# read the configuraiton file
config_parser = parse_input.ModelConfigParser("../../examples/test1/config.txt")
config_values = config_parser.parse_config()

# read the labels and parameterized files
labels = np.load(os.path.join(config_values["output directory"], 'labels.npy'), allow_pickle=True)
with open(os.path.join(config_values["output directory"], 'parameterized_models.pickle'), 'rb') as f:
    parameterized_models = pickle.load(f)


## Read empirical data, and convert it into a numpy array.

First, generate our data processor. Then, we convert our folder of fasta files into a numpy array. We will keep the same number of sites that we kept for simulated data. Missing data will be encoded as -1 (any sites in the alignment other than A, T, C, and G will be converted to -1).

In [3]:
# create our data processor, and convert our fastas to a numpy array
data_processor = process_empirical.DataProcessor(parameterized_models, config=config_values)
empirical_array = data_processor.fasta_to_numpy()
print(empirical_array.shape) # print the shape of our array: (haploid samples, SNPs).

(30, 1038)


## Choose projection for site-frequency-spectrum

SFS cannot be generated from datasets that include missing data. To circumvent this, we use a downsampling approach such as that described in Satler and Carstens (2017, Molecular Ecology, doi: 10.1111/mec.14137.) We must choose thresholds for each populations (i.e., the minumum number of individuals that must be sampled for a SNP to be used.) To help with this, we use the function find_downsampling from the class DataProcessor. This function generates a dictionary that holds the number of SNPs that meet each threshold.

Since our data should be phased, and we will simulate diploid individuals, we will only consider multiples of 2.

In [4]:
# generate dictionary with the number of SNPs at different sampling thresholds
empirical_2d_sfs_sampling = data_processor.find_downsampling(empirical_array)

# print threshold, SNP combos for thresholds with at least 1030 snps
minspns = 1030
min1000 = {key: value for key, value in empirical_2d_sfs_sampling.items() if value >= minspns}
print(min1000)


{(2, 2, 2): 1038, (2, 2, 4): 1038, (2, 2, 6): 1038, (2, 4, 2): 1038, (2, 4, 4): 1038, (2, 4, 6): 1038, (2, 6, 2): 1035, (2, 6, 4): 1035, (2, 6, 6): 1035, (4, 2, 2): 1038, (4, 2, 4): 1038, (4, 2, 6): 1038, (4, 4, 2): 1038, (4, 4, 4): 1038, (4, 4, 6): 1038, (4, 6, 2): 1035, (4, 6, 4): 1035, (4, 6, 6): 1035, (6, 2, 2): 1038, (6, 2, 4): 1038, (6, 2, 6): 1038, (6, 4, 2): 1038, (6, 4, 4): 1038, (6, 4, 6): 1038, (6, 6, 2): 1035, (6, 6, 4): 1035, (6, 6, 6): 1035, (8, 2, 2): 1037, (8, 2, 4): 1037, (8, 2, 6): 1037, (8, 4, 2): 1037, (8, 4, 4): 1037, (8, 4, 6): 1037, (8, 6, 2): 1034, (8, 6, 4): 1034, (8, 6, 6): 1034}


## Down-project the SFS

Now, we must choose a threshold. I will choose (9,6,7), as it is the largest combination of thresholds that gives us more than 1030 SNPs.

In [8]:
# build 10 replicates of the 2d SFS
empirical_2d_sfs = data_processor.numpy_to_2d_sfs(empirical_array, downsampling={"A":8, "B":6, "C":6}, replicates = 10)

# build 10 replicates of the mSFS
empirical_msfs = data_processor.numpy_to_msfs(empirical_array, downsampling={"A":8, "B": 6, "C":6}, replicates = 10)

# pickle dump the SFS
with open(os.path.join(config_values["output directory"], 'empirical_2d_sfs.pickle'), 'wb') as f:
    pickle.dump(empirical_2d_sfs, f)

# pickle dump the SFS
with open(os.path.join(config_values["output directory"], 'empirical_msfs.pickle'), 'wb') as f:
    pickle.dump(empirical_msfs, f)

## Save the SFS

